In [20]:
import pandas as pd
import numpy as np

In [3]:
dir_raw_data = r""
dir_fine_data = r""
df = pd.read_csv(dir_raw_data)

In [4]:
df

,Unnamed: 0,text,legislature,group,name,gender,monthly_intervals,year,mistral_large
0,0,De acuerdo con los datos que obran en la Secre...,1,NaN,NaN,NaN,1979-03,1979,0
1,1,"Señoras y señores Diputados, se abre la sesión...",1,CP,Fraile Poujade,Male,1979-03,1979,0
2,2,sEENERAL DEL CONGRESO DE LOS DIPUTADOS (Rubio ...,1,NEUTRAL,Secretario,NaN,1979-03,1979,0
3,3,El artículo que hace referencia a este acto es...,1,NaN,NaN,NaN,1979-03,1979,0
4,4,Según la relación que consta en la Secretaría ...,1,CP,Fraile Poujade,Male,1979-03,1979,0
...,...,...,...,...,...,...,...,...,...
432296,432296,La estamos estudiando. Muchas gracias.\n,15,VOX,Hoces Íñiguez,Male,2024-05,2024,0
432297,432297,De acuerdo. Votamos en un minuto. (Pausa). TOM...,15,NEUTRAL,Presidenta,NaN,2024-05,2024,0
432298,432298,"Señoras y señores diputados, vamos a proceder ...",15,NEUTRAL,Presidenta,NaN,2024-05,2024,0
432299,432299,"En consecuencia, se toma en consideración esta...",15,NEUTRAL,Presidenta,NaN,2024-05,2024,0


### Clasificación pro grupos

Entre todos los grupos presentes en los datos del Congreso, hacemos la siguiente división.

In [6]:
groups = {
    "PP": set(["PN-PP", "PP-EU", "PP-PAR", "AP", "PP-CG", "CP", "PP", "UPM", "PP-FORO", "UPN"]),
    "PSOE": set(["PsdeG-PSOE", "PSOE-A", "PSE-EE","PSC(PSC-PSOE)","PSOEdeAndalucía","PSC-PSOE","PSOE-progresistas","PSG-PSOE","PSE-EE-PSOE","PSE-EE (PSOE)","Pse-EE(PSOE)","PSE-PSOE","PSE-EE(PSOE)","PSOE-NCa","PSN-PSOE","PSIB-PSOE","PSA","PSdG-PSOE","PSdeG-PSOE-progresistas","PSOE"]),
    "VOX": set(["Vox", "VOX"]),
    "IU-PODEMOS": set(["IU-CA", "IU-EUPV", "PODEMOS", "PCE", "IU-CM", "IU-EB", "IU", "UP", "SUMAR", "IU-LV-CA", "MÁS PAÍS-EQUO", "IU-IX", "IU-LV", "IZQ-PLU", "IU-A"])
}

In [ ]:
def agrupar(subgrupo):
    for grupo in groups:
        if subgrupo in groups[grupo]:
            return grupo
    return subgrupo

In [19]:
df["group"] = df["group"].apply(agrupar)

In [ ]:
## Vox entra en el Congreso en diciembre de 2018. Las intervenciones de VOX previas son de diputados que han cambiado de partido. Las desestimamos.

for index, row in df.iterrows():
    if row["group"] == "VOX" and row["year"] < 2018:
        df.at[index, "group"] = np.nan

In [39]:
df = df.drop(df[(df["group"] == np.nan) | (df["group"] == "NEUTRAL") | ~ (df["group"].isin(groups))].index)

### Final

In [32]:
df = df.drop(columns=["monthly_intervals", "name", "legislature", "Unnamed: 0"])

In [43]:
df

,text,group,gender,year,mistral_large
1,"Señoras y señores Diputados, se abre la sesión...",PP,Male,1979,0
4,Según la relación que consta en la Secretaría ...,PP,Male,1979,0
5,"Siguiendo el orden del día, ruego al señor Esp...",PSOE,Male,1979,0
6,Señoras y señores Diputados. en nombre de la M...,PSOE,Male,1979,0
7,"Al reanudarse la sesión, procedería dar lectur...",PSOE,Male,1979,0
...,...,...,...,...,...
432286,"Muchas gracias, señora presidenta. Buenas tard...",VOX,Male,2024,15
432288,"Muchas gracias, presidenta. Buenas tardes, señ...",PP,Male,2024,30
432292,"Buenas tardes, noches ya, señorías. Probableme...",IU-PODEMOS,Male,2024,22
432294,"Gracias, presidenta. Buenas tardes, señorías. ...",PSOE,Female,2024,9


In [41]:
df.to_csv(dir_fine_data, index=False)